# Exploratory Data Analysis

- https://dacon.io/competitions/official/235875/codeshare/4520?page=1&dtype=recent
의 EDA과정을 참고하였음.
- https://wikidocs.net/92961
의 한국어 맞춤법 교정 과정을 참조하였음.

## Import Packages

In [ ]:
# !pip3 install pandas
# !pip3 install matplotlib
# !pip3 install torch

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

## Load Data

In [2]:
# PATH = '~/Desktop/VSC/DACON/text_regression_task/dataset'   # MAC
PATH = './dataset/' # Windows

train_data = pd.read_csv(os.path.join(PATH, 'train.csv'), encoding='utf-8')     # 25000
test_data = pd.read_csv(os.path.join(PATH, 'test.csv'), encoding='utf-8')       # 25000

In [ ]:
"""
id      : 평점 인덱스
reviews : 리뷰
target  : 평점
"""

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

print(train_data.head(10), end='\n\n')
print(test_data.head(10))

## Train, Test Data Exploration


In [ ]:
print(train_data.info())
print()
print(test_data.info())

In [ ]:
# 데이터 label별 균형 확인
print('Train Label: ', train_data['target'].value_counts(), sep='\n', end='\n\n' )

Train data의 Label 분류 결과<br>
5점이 10000개<br>
4점이 2500개<br>
3점이 0개<br>
2점이 8000개<br>
1점이 4500개<br>

--> label 불균형 존재한다.

In [ ]:
# 결측지 확인
print('Train Null: ', train_data.isnull().sum(), sep='\n', end='\n\n')
print('Test Null: ', test_data.isnull().sum(), sep='\n')

In [ ]:
# label 분포
feature = train_data['target']

plt.figure(figsize = (10, 7.5))
plt.title('Label Counting', fontsize=20)

temp = feature.value_counts()
## temp.values -> 5 2 1 4
plt.bar(temp.keys(), temp.values, width=0.5, color='b', alpha=0.5)
plt.text(4.85, temp.values[0]+20, s=temp.values[0]) # 5
plt.text(1.85, temp.values[1]+20, s=temp.values[1]) # 2
plt.text(0.85, temp.values[2]+20, s=temp.values[2]) # 1
plt.text(3.85, temp.values[3]+20, s=temp.values[3]) # 4

plt.xticks(temp.keys(), fontsize=12)    # x축값
plt.show()

## Grammar Correction

- Py-Hanspell 이 띄어쓰기와 철자 둘 다 교정하기에, 우선 이를 전처리기로 사용

### Library Testing

In [ ]:
# PyKoSpacing 띄어쓰기 교정 라이브러리
# str형태만 입력으로 받을 수 있어, 병렬처리는 안된다.
# !pip3 install git+https://github.com/haven-jeon/PyKoSpacing.git

# Py-Hanspell 맞춤법 교정 라이브러리
# !pip3 install git+https://github.com/ssut/py-hanspell.git

#### PyKoSpacing : 띄어쓰기

In [ ]:
# sent = '해당 라이브러리는 전희원 님이 개발하신 띄어쓰기가 되어있지 않은 문장을 띄어쓰기 된 문장으로 변환해주는 패키지 이다. 대용량 코퍼스를 학습하여 만들어진 띄어쓰기 딥러닝 모델로 준수한 성능을 가지고있다.'
# new_sent = sent.replace(' ', '')
# print(new_sent)

In [ ]:
# from pykospacing import Spacing
# spacing = Spacing()
# kospacing_sent = spacing(new_sent)

# print(kospacing_sent)

In [ ]:
# train_data['reviews'][80:100]
# # train_data['reviews'][21]

In [ ]:
# sample_input = train_data['reviews'][21]
# sample_result = spacing(sample_input)
# print( sample_input, '\n\n', sample_result)

#### Py-Hanspell : 맞춤법 교정

In [ ]:
from hanspell import spell_checker

sample_input = train_data['reviews'][21]
sample_result_spell = spell_checker.check(sample_input)
print( sample_input, '\n\n', sample_result_spell.checked)

In [ ]:
# 병렬처리 테스트
pi_ = ['앙녕', '앙녕하세요']
output_spell = spell_checker.check(pi_)
# print(output_spell)
# list 로 병렬처리 된다.

## 데이터프레임 객체도 병렬처리 지원하는지?
pd_pi_ = pd.DataFrame(pi_)
output_spell_pd = spell_checker.check(pd_pi_)
# print(output_spell_pd)
# list만 되는듯하다.

우선, 맞춤법교정기가 문법과 띄어쓰기까지 교정한다 가정하고 pass

### Outlier

In [ ]:
print(
    train_data.iloc[58],'\n\n',
    train_data.iloc[93]
)

# Train, Test data Preprocessing - Grammer Correction

#### 불용어 처리

In [ ]:
# wrong_idx = [932,3093,4933,7784,9457,13134,15404,17440,17669,18047,19745,22504,24383]   # "&" 를 처리했음.

# for i in wrong_idx:
#     print('before -- ', train_data['reviews'][i])
#     temp = train_data['reviews'][i].replace("&","")
#     train_data['reviews'][i] = temp
#     print('after -- ',train_data['reviews'][i])
#     print()
    

### 데이터 전체 맞춤법 교정

In [5]:
# train_data 맞춤법 전처리
from hanspell import spell_checker

train_refined_reviews = []
for i in tqdm(range(0,25000),desc='Preprocessing... This might take a while... '):
    try:
        temp = spell_checker.check(train_data.iloc[i]['reviews'])
        train_refined_reviews.append(temp.checked)
    except:
        temp = train_data['reviews'][i].replace("&","")
        temp = spell_checker.check(temp)
        train_refined_reviews.append(temp.checked)

Preprocessing... This might take a while... : 100%|██████████| 25000/25000 [29:14<00:00, 14.25it/s] 


In [6]:
len(train_refined_reviews)    # 25000
train_data['reviews'] = train_refined_reviews
del train_refined_reviews

In [7]:
# 저장
train_data.to_csv('./dataset/preprocessed_train.csv', mode='w')
del train_data

In [8]:
# test_data 맞춤법 전처리
from hanspell import spell_checker

test_refined_reviews = []
for i in tqdm(range(0,25000),desc='Preprocessing... This might take a while...'):
    try:
        temp = spell_checker.check(test_data.iloc[i]['reviews'])
        test_refined_reviews.append(temp.checked)
    except:
        temp = test_data['reviews'][i].replace("&","")
        temp = spell_checker.check(temp)
        test_refined_reviews.append(temp.checked)

Preprocessing... This might take a while...: 100%|██████████| 25000/25000 [29:23<00:00, 14.17it/s]  


In [10]:
test_data['reviews'] = test_refined_reviews
del test_refined_reviews

In [12]:
test_data.to_csv('./dataset/preprocessed_test.csv', mode='w')
del test_data